In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from utils import create_sequences

# import all data
df = pd.read_csv('data/upload_DJIA_table.csv', parse_dates=['Date'], index_col='Date')
df = df[['Close']]
df = df.sort_index()

train_data = df[:'2014']
test_data = df['2015':]

bert_embeddings = np.load('bert_embeddings.npy')

bert_embeddings_train = bert_embeddings[:train_data.shape[0]].reshape(-1, 768)
bert_embeddings_test = bert_embeddings[train_data.shape[0]:].reshape(-1, 768)
    

X_train_emb, X_train_pr, y_train = create_sequences(train_data, 60, bert_embeddings_train)
X_test_emb, X_test_pr, y_test = create_sequences(test_data, 60, bert_embeddings_test)


print("Train shapes: ", X_train_emb.shape, X_train_pr.shape, y_train.shape)
print("Test shapes: ", X_test_emb.shape, X_test_pr.shape, y_test.shape)

# convert to torch dataset
dataset_train = TensorDataset(
    torch.from_numpy(X_train_emb).float(),
    torch.from_numpy(X_train_pr).float(),
    torch.from_numpy(y_train).float()
)
dataset_test = TensorDataset(
    torch.from_numpy(X_test_emb).float(),
    torch.from_numpy(X_test_pr).float(),
    torch.from_numpy(y_test).float()
)

dataloader_train = DataLoader(dataset_train, batch_size=32, shuffle=True)
dataloader_test = DataLoader(dataset_test, batch_size=32, shuffle=True)

Train shapes:  (1551, 60, 768) (1551, 60) (1551,)
Test shapes:  (318, 60, 768) (318, 60) (318,)


In [2]:
from models import StockPredictor

# Model parameters
embedding_dim = 768  # Size of BERT embeddings
price_dim = 1        # Each stock price is a single number
hidden_dim = 128
num_layers = 2

model = StockPredictor(embedding_dim, price_dim, hidden_dim, num_layers)


In [3]:
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim


# Training parameters
num_epochs = 10
learning_rate = 0.001

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
model.train()
for epoch in range(num_epochs):
    for batch_bert, batch_price, batch_y in dataloader_train:
        optimizer.zero_grad()
        outputs = model(batch_bert, batch_price.unsqueeze(-1))
        loss = criterion(outputs.squeeze(), batch_y)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}, MAE: {loss.item()**0.5}")


/Users/ignatiocalvin/anaconda3/envs/dowJones/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch 1, Loss: 178201392.0, MAE: 13349.209414793073
Epoch 2, Loss: 146288288.0, MAE: 12094.969532826446
Epoch 3, Loss: 129430936.0, MAE: 11376.771774101826
Epoch 4, Loss: 61263784.0, MAE: 7827.118499166855
Epoch 5, Loss: 35276108.0, MAE: 5939.36932678883
Epoch 6, Loss: 9780979.0, MAE: 3127.455675145533
Epoch 7, Loss: 11902432.0, MAE: 3449.9901449134604
Epoch 8, Loss: 5983936.0, MAE: 2446.2084947935246
Epoch 9, Loss: 6555874.0, MAE: 2560.4441021041644
Epoch 10, Loss: 6831317.5, MAE: 2613.6789206021463


In [7]:
import torchmetrics

mse = torchmetrics.MeanSquaredError()
model.eval()
with torch.no_grad():
    for batch_bert, batch_price, batch_y in dataloader_test:
        outputs = model(batch_bert, batch_price.unsqueeze(-1))
        mse(outputs.squeeze(), batch_y)

print("Test MSE: ", mse.compute())
print("Test MAE: ", mse.compute()**0.5)   

Test MSE:  tensor(23406206.)
Test MAE:  tensor(4837.9961)
